In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper= api_wrapper)

In [3]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.geeksforgeeks.org/machine-learning/machine-learning/")
doc = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)
doc_trans = text_splitter.split_documents(doc)
from langchain.schema import Document
if not hasattr(doc_trans[0], 'page_content'):
    doc_trans = [Document(page_content=chunk) for chunk in doc_trans]

In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\jaski\AppData\Local\Temp\ipykernel_30260\3415835077.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\jaski\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\jaski\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will

In [41]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(doc_trans,embeddings)

In [42]:
db.similarity_search("what is machine learning")

[Document(metadata={'title': 'Machine Learning Tutorial - GeeksforGeeks', 'language': 'en-US', 'source': 'https://www.geeksforgeeks.org/machine-learning/machine-learning/', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.'}, page_content='Machine learning is a branch of\xa0Artificial Intelligence\xa0that focuses on developing models and algorithms that let computers learn from data without being explicitly programmed for every task. In simple words, ML teaches the systems to think and understand like humans by learning from the data.Machine Learning is mainly divided into three core types: Supervised, Unsupervised and Reinforcement Learning along with two additional types, Semi-Supervised and Self-Supervised Learning.Supervised Learning: Trains models on labeled data to

In [43]:
retriver = db.as_retriever()

In [44]:
from langchain.tools.retriever import create_retriever_tool
ml_tool = create_retriever_tool(retriver,"machine_learning","this tool will provide all the information which is available about machine learning")

In [10]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
api_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper)

In [11]:
tools = [wiki_tool,ml_tool,arxiv_tool]

In [14]:
from dotenv import load_dotenv
load_dotenv()
import os
google_api = os.getenv('google_api')

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model = 'gemini-1.5-flash',
    google_api_key = google_api
)

In [17]:
from langchain.agents import AgentType

In [33]:
from langchain_core.prompts import ChatPromptTemplate
promt = ChatPromptTemplate.from_messages([('human',"{input}")])

In [49]:
from langchain.agents import initialize_agent
agent = initialize_agent(tools,llm,agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         prompt = promt)

In [50]:
agent.invoke({'input':"salman khan wiki page"})

ValueError: Missing some input keys: {'', 'page_content'}